In [1]:
!pip install keras

In [2]:
# The essentials
import pandas as pd
import numpy as np

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Progress bars
from tqdm import tqdm_notebook

import pickle
from time import time

from keras import models, layers, optimizers, callbacks

# Access our Google Drive
from google.colab import drive

Using TensorFlow backend.


In [0]:
import tensorflow as tf
import keras

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [4]:
drive.mount('/content/drive', force_remount=True)
!ls "/content/drive/My Drive/Rinse Over Run"

Mounted at /content/drive
all_train_preds_per_phase.p
baseline_features_with_preds_per_phase.csv
baseline_model_per_nunique_phases.csv
predictions_machine_405.csv
test_features_14.csv
test_features_15.csv
test_features_1.csv
test_features_2.csv
test_features_3.csv
test_features_6.csv
test_features_7.csv
test_features_8.csv
test_features_per_phase_14.csv
test_features_per_phase_15.csv
test_features_per_phase_1.csv
test_features_per_phase_2.csv
test_features_per_phase_3.csv
test_features_per_phase_6.csv
test_features_per_phase_7.csv
test_features_per_phase_8.csv
test_preds_per_phase.p
test_values.csv
train_features_14.csv
train_features_15.csv
train_features_1.csv
train_features_2.csv
train_features_3.csv
train_features_6.csv
train_features_7.csv
train_features_8.csv
train_features_adv_14.csv
train_features_adv_15.csv
train_features_adv_1.csv
train_features_adv_2.csv
train_features_adv_3.csv
train_features_adv_6.csv
train_features_adv_7.csv
train_features_adv_8.csv
train_features_per_pha

In [0]:
def filter_data(raw_data, processes, phases, columns):
    filtered_data = raw_data[(raw_data['process_id'].isin(processes)) & 
                             (raw_data['phase'].isin(phases))][['process_id', 'timestamp'] + columns]
    filtered_data = filtered_data.reset_index(drop=True)
    return filtered_data
  
def resample_fixed(df, n_new):
    n_old, m = df.values.shape
    mat_old = df.values
    mat_new = np.zeros((n_new, m))
    x_old = np.linspace(df.index.min(), df.index.max(), n_old)
    x_new = np.linspace(df.index.min(), df.index.max(), n_new)

    for j in range(m):
        y_old = mat_old[:, j]
        y_new = np.interp(x_new, x_old, y_old)
        mat_new[:, j] = y_new

    return pd.DataFrame(mat_new, index=x_new, columns=df.columns)

def get_time_series(data, processes, cols, mask=0, max_len=1000, masking=False):
    all_ts_values = []
    
    for col in ts_cols:
      data[col] = data[col].astype(float)
      data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())
      
    for ix, process in enumerate(processes):
        filtered_data = data[data['process_id'] == process]
        filtered_data = filtered_data.sort_values(by='timestamp')
        filtered_data = filtered_data.reset_index(drop=True)
        filtered_data = filtered_data[cols].astype(float)
        for col in filtered_data.columns:
          if min(filtered_data[col]) < 0:
            filtered_data[col] += (abs(min(filtered_data[col])) + 1)
        if len(filtered_data) > max_len:
          filtered_data['ix'] = filtered_data.index // (len(filtered_data)/max_len)
          filtered_data = filtered_data.groupby('ix').mean()
          ts_values = filtered_data[ts_cols].values
        elif not masking and len(filtered_data) < max_len:
          ts_values = resample_fixed(filtered_data, max_len).values
          
        if masking:
            ts_values = filtered_data.values
            ts_values = np.vstack((ts_values, mask*np.ones((max_len - len(ts_values), ts_values.shape[1]))))
          
        all_ts_values.append(ts_values)

    return np.array(all_ts_values)
  
def window_time_series(data, labels, processes, cols, window_size=25):
    windows = []
    y = []
    
    for col in ts_cols:
      data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())
    
    for ix, process in enumerate(processes):
        filtered_data = data[data['process_id'] == process]
        filtered_data = filtered_data.sort_values(by='timestamp')
        filtered_data = filtered_data.reset_index(drop=True)
        filtered_data = filtered_data[cols].astype(float)
        
        ts_values = filtered_data[ts_cols].values
        
        for i in range(len(ts_values), window_size, -window_size):
            windows.append(ts_values[i - window_size:i])
            y.append(labels.loc[process])
            
    return np.array(windows), np.array(y)

In [25]:
PROCESS_COMBINATION = 3

phases = {
    15: ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid'],
    3:  ['pre_rinse', 'caustic'],
    7:  ['pre_rinse', 'caustic', 'intermediate_rinse'],
    1:  ['pre_rinse'],
    8:  ['acid'],
    2:  ['caustic'],
    6:  ['caustic', 'intermediate_rinse'],
    14: ['caustic', 'intermediate_rinse', 'acid'],
}

ts_cols = [
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    'supply_pump',
    'supply_pre_rinse',
    'supply_caustic',
    'return_caustic',
    'supply_acid',
    'return_acid',
    'supply_clean_water',
    'return_recovery_water',
    'return_drain',
    'object_low_level'
]

"""
'tank_level_pre_rinse',
'tank_level_caustic',
'tank_level_acid',
'tank_level_clean_water',
'tank_temperature_pre_rinse',
'tank_temperature_caustic',
'tank_temperature_acid',
'tank_concentration_caustic',
'tank_concentration_acid',
'supply_pump',
'supply_pre_rinse',
'supply_caustic',
'return_caustic',
'supply_acid',
'return_acid',
'supply_clean_water',
'return_recovery_water',
'return_drain',
'object_low_level'
"""

"\n'tank_level_pre_rinse',\n'tank_level_caustic',\n'tank_level_acid',\n'tank_level_clean_water',\n'tank_temperature_pre_rinse',\n'tank_temperature_caustic',\n'tank_temperature_acid',\n'tank_concentration_caustic',\n'tank_concentration_acid',\n'supply_pump',\n'supply_pre_rinse',\n'supply_caustic',\n'return_caustic',\n'supply_acid',\n'return_acid',\n'supply_clean_water',\n'return_recovery_water',\n'return_drain',\n'object_low_level'\n"

In [26]:
train_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_values.csv', index_col=0, parse_dates=['timestamp'])
labels = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_labels.csv', index_col=0)
train_features = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_features_{}.csv'.format(PROCESS_COMBINATION), index_col='process_id')

train_procs = set(pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_features_adv_{}.csv'.format(PROCESS_COMBINATION), index_col='process_id').index)
val_procs = set(pd.read_csv('/content/drive/My Drive/Rinse Over Run/val_features_adv_{}.csv'.format(PROCESS_COMBINATION), index_col='process_id').index)

#train_procs = np.random.choice(train_procs, replace=False, size=250)
#val_procs = np.random.choice(val_procs, replace=False, size=250)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
#train_df.groupby('process_id')['timestamp'].count().max()

In [39]:
filtered_data = filter_data(train_df, train_procs, phases[PROCESS_COMBINATION], ts_cols)
#train_windows, train_labels = window_time_series(filtered_data, labels, train_procs, ts_cols)
#train_labels = np.log(train_labels)
time_series = get_time_series(filtered_data, train_procs, ts_cols, masking=True)
train_labels = np.log(labels.loc[train_procs].values)
#train_mean = np.mean(train_labels)
#train_std = np.std(train_labels)
#train_labels = (train_labels - train_mean) / train_std

filtered_data = filter_data(train_df, val_procs, phases[PROCESS_COMBINATION], ts_cols)
#val_windows, val_labels = window_time_series(filtered_data, labels, val_procs, ts_cols)
#val_labels = np.log(val_labels)
val_time_series = get_time_series(filtered_data, val_procs, ts_cols, masking=True)
val_labels = np.log(labels.loc[val_procs].values)
#val_labels = (val_labels - train_mean) / train_std

print(time_series.shape, val_time_series.shape)

(4269, 1000, 25) (474, 1000, 25)


In [16]:
def custom_mape_np(approxes, targets):
    return np.mean(np.abs(np.subtract(approxes, targets)) / np.maximum(np.abs(targets), 290000))

print(model.predict(val_windows) * train_std + train_mean)
custom_mape_np(np.array(model.predict(val_windows) * train_std + train_mean), val_labels * train_std + train_mean)

NameError: ignored

In [0]:
import keras.backend as K
def custom_mape_exp(y_true,y_pred):
    y_true = K.exp(y_true)
    y_pred = K.exp(y_pred)
    
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true),
                                            290000,
                                            None))
    return 100. * K.mean(diff, axis=-1)
  
def custom_smape_exp(y_true,y_pred):
    y_true = K.exp(y_true)
    y_pred = K.exp(y_pred)
  
    return K.mean(K.abs(y_pred - y_true) / (y_pred + y_true))
  
def custom_mape(y_true,y_pred):
    return K.mean(K.abs(y_pred - y_true) / K.maximum(K.abs(y_true), np.log(290000)))

In [0]:
class NBatchLogger(keras.callbacks.Callback):
    def __init__(self, test_data, display=50):
        self.display = display
        self.test_data = test_data
        self.seen = 0
    def on_batch_end(self, epoch, logs={}):
        self.seen += 1
        if self.seen % self.display == 0:
            x, y = self.validation_data
            val_loss = self.model.evaluate(x, y, verbose=0)
            print('\n[Epoch #{}] train loss {} - val loss {}\n'.format(self.seen, logs.get('loss'), val_loss)) 

In [44]:
model = models.Sequential()

model.add(layers.Conv1D(1024, 7, activation='relu', input_shape=(1000, len(ts_cols))))
model.add(layers.BatchNormalization())
#model.add(layers.Dropout(0.3))
model.add(layers.MaxPooling1D(8))

model.add(layers.Conv1D(1024, 5, activation='relu'))
model.add(layers.BatchNormalization())
#model.add(layers.Dropout(0.3))
model.add(layers.MaxPooling1D(8))

model.add(layers.Conv1D(1024, 3, activation='relu'))
model.add(layers.BatchNormalization())
#model.add(layers.Dropout(0.3))
model.add(layers.MaxPooling1D(8))

model.add(layers.Flatten())
#model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.Adam(lr=0.001), loss='mean_absolute_percentage_error')

print(model.summary())

model.fit(time_series, train_labels, batch_size=128, verbose=2,
          epochs=1000, validation_data=(val_time_series, val_labels),
          callbacks=[callbacks.EarlyStopping(patience=100)])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_40 (Conv1D)           (None, 994, 1024)         180224    
_________________________________________________________________
batch_normalization_40 (Batc (None, 994, 1024)         4096      
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 124, 1024)         0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 120, 1024)         5243904   
_________________________________________________________________
batch_normalization_41 (Batc (None, 120, 1024)         4096      
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 15, 1024)          0         
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 13, 1024)          3146752   
__________

KeyboardInterrupt: ignored

In [0]:
def custom_mape_np(approxes, targets):
    return np.mean(np.abs(np.subtract(approxes, targets)) / np.maximum(np.abs(targets), 290000))

custom_mape_np(np.exp(model.predict(val_time_series)), np.exp(val_labels))

In [0]:
model = models.Sequential()

model.add(layers.Masking(0, name="input", input_shape=(25, len(ts_cols))))
model.add(layers.GRU(128, return_sequences=False, name='lstm1')) #layers.Bidirectional()
model.add(layers.Dense(64))
model.add(layers.Dense(1, name='output'))

print(model.summary())

model.compile(optimizers.Adam(), loss='mean_absolute_percentage_error')

#model.add(layers.TimeDistributed(layers.Dense(1)))
#model.add(layers.Dropout(0.1, name='drop1'))
#model.add(layers.Dense(64, activation='relu', name='dense1'))
#model.add(layers.Dropout(0.1, name='drop1'))
#model.add(layers.Dense(64, activation='relu', name='dense2'))
#model.add(layers.Dense(64, activation='relu', name='dense3'))
#model.add(layers.Dropout(0.15, name='drop3'))

model.fit(train_windows, train_labels, batch_size=256, 
          epochs=1000, validation_data=(val_windows, val_labels))
          #callbacks=[callbacks.EarlyStopping(patience=25)])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Masking)              (None, 25, 2)             0         
_________________________________________________________________
lstm1 (GRU)                  (None, 128)               50304     
_________________________________________________________________
dense_52 (Dense)             (None, 64)                8256      
_________________________________________________________________
output (Dense)               (None, 1)                 65        
Total params: 58,625
Trainable params: 58,625
Non-trainable params: 0
_________________________________________________________________
None
Train on 889 samples, validate on 1390 samples
Epoch 1/1000
889/889 [==============================] - 10s 11ms/step - loss: 107.1989 - val_loss: 100.7409
Epoch 2/1000
889/889 [==============================] - 0s 487us/step - loss: 104.7768 - val_loss: 100.2255
Ep

In [0]:
"""
model = models.Sequential()

model.add(layers.Masking(-100, name="input", input_shape=(None, len(ts_cols))))
model.add(layers.BatchNormalization())
model.add(layers.LSTM(128, return_sequences=True, name='lstm1'))
model.add(layers.Dropout(0.1))
model.add(layers.TimeDistributed(layers.Dense(1, name='output')))

print(model.summary())

model.compile(optimizers.Adam(), loss=custom_mape_exp)
"""

In [0]:
"""
from keras.layers.core import Permute

def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    a = Permute((2, 1))(inputs)
    a = Reshape((25, 250))(a)
    a = layers.Dense(25, activation='softmax')(inputs)
    output_attention_mul = layers.merge.multiply([inputs, a], name='attention_mul')
    return output_attention_mul

inputs = layers.Input(shape=(250, len(ts_cols)))
attention_mul = attention_3d_block(inputs)
mask = layers.Masking(-100)(attention_mul)
lstm_out = layers.LSTM(128, return_sequences=False)(mask)
dense1 = layers.Dense(64, activation='relu')(lstm_out)
dense2 = layers.Dense(32, activation='relu')(dense1)
output = layers.Dense(1)(dense2)
model = models.Model(input=[inputs], output=output)

print(model.summary())

model.compile(optimizers.Adam(), loss=custom_mape_exp)
"""

In [0]:
#(4272, 250, 25) (474, 250, 25)  #Reshape the (?, 12,1000) tensor to (?, 1000, 12, 1) tensor.

#time_series_reshaped = np.reshape(time_series, (4272, 25, 250, 1))
#time_series_val_reshaped = np.reshape(val_time_series, (474, 25, 250, 1))

In [0]:
model.fit(time_series, train_labels, batch_size=64, epochs=1000, validation_data=(val_time_series, val_labels))

In [0]:
"""
{15: 0.25369314638650514, 3: 0.3485206977395646, 7: 0.27274180323993563, 
 1: 0.3258186511871589, 8: 0.49481063651450297, 2: 0.3888194424409063, 
 6: 0.28262367909221875, 14: 0.28390152064193996}
Expected LB MAPE = 0.31390297272100504
"""